# Trilateration: Analysis

Given a set of RTT samples annotated by probe and anchor ID, and metadata about anchors including location, determine the linear fit between 

### Needful things

First, imports, utility functions, etc. required for analysis, and thaw out the datastore

In [2]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import math

from collections import namedtuple

In [3]:
with pd.HDFStore('rtt.h5') as store:
    anchor_df = store['anchor_df']
    probe_df = store['probe_df']
    rtt_df = store['rtt_df']

### Distance calculation

Given a pair of latitude and longitude, determine distance (along a spherical approximation of the Earth) in kilometers.


In [4]:
def on_unit_sphere(lat1, long1, lat2, long2):
 
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta', phi')
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
 
    return arc

def earth_surface_km(lat1, long1, lat2, long2):
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return on_unit_sphere(lat1, long1, lat2, long2) * 6371

Add location and distance to the RTT dataframe

In [5]:
loc_by_aid = anchor_df.loc[:,('lon','lat')]
loc_by_aid.columns = ['alon','alat']
loc_by_pid = probe_df.loc[:,('lon','lat')]
loc_by_pid.columns = ['plon','plat']

dist_df = rtt_df.join(loc_by_aid, on="aid").join(loc_by_pid, on="pid")

In [6]:
dist_df['km'] = dist_df.apply(lambda x: earth_surface_km(x["alat"], x["alon"], x["plat"], x["plon"]), axis=1)

## Next steps

- Find linear fit for RTT/distance (see below)
- Show RTTs for each distance bin and distances for each RTT bin as boxplots
- See how many samples per pair it takes before rvalue stops rising (run dataprep on a six hour chunk?
- Generate most likely distance for each RTT bin (in ms?)?
- Attempt actual 3D trilateration (see https://gis.stackexchange.com/questions/66/trilateration-using-3-latitude-and-longitude-points-and-3-distances) and generalize to n-lateration, calculate errors. 

In [7]:
scipy.stats.linregress(dist_df['km'],dist_df['rtt']/2)

LinregressResult(slope=9.159571264477302, intercept=13660.048580854054, rvalue=0.81515565133594392, pvalue=0.0, stderr=0.0064019562838746819)

Now minimize RTT and see if we can do any better

In [8]:
scipy.stats.linregress(dist_df.groupby(['aid','pid'])['km'].min(), dist_df.groupby(['aid','pid'])['rtt'].min()/2)

LinregressResult(slope=9.0061821825123314, intercept=11417.138781831905, rvalue=0.87169878724481187, pvalue=0.0, stderr=0.043200951721642844)

In [9]:
(1e9 / 9.0061821825123314) / 299792458

0.3703723602725325